# 第十二讲：$k$-means算法、高斯混合模型及最大期望算法

现在我们开始进入机器学习课程新的一章——无监督学习算法的应用。

## $k$-means聚类算法（$k$-means clustering algorithm）

在聚类问题（clustering problem）中，对于给定的训练集$\left\{x^{(1)},\cdots,x^{(m)}\right\}$，并且希望对样本分组，并找到能够使样本“紧密聚集”在一起的簇（clusters）。这里的$x^{(i)}\in\mathbb{R}^n$像原来一样，但是没有$y^{(i)}$。因此，这是一个无监督学习问题。

$k$-means聚类算法的过程如下：

1. 随机的初始化**聚类质心（cluster centroids）**$\mu_1,\mu_2,\cdots,\mu_k\in\mathbb{R}^n$；
2. 重复直到收敛

  `{`
  
  * 对每一个样本$i$，令$c^{(i)}:=\mathrm{arg}\displaystyle\operatorname*{min}_j\left\lVert x^{(i)}-\mu_j\right\rVert^2$
  * 对每一个簇$j$，令$\mu_j:=\frac{\sum_{i=1}^m1\left\{c^{(i)}=j\right\}x^{(i)}}{\sum_{i=1}^m1\left\{c^{(i)}=j\right\}}$
      
  `}`

在这个算法中，参数$k$表示我们想要聚类的簇的数量，$\mu_j$表示我们当前对于簇质心位置的猜测。为了初始化簇质心（即算法第一步），我们通常随机选择$k$个训练样本，然后令$k$个质心等于这$k$个训练样本。（方法不唯一，其他初始化方法也是可以的。）

算法的内层循环重复这两步：(i)将每个训练样本$x^{(i)}$分配给距离它们最近的簇质心$\mu_j$，(ii)移动每个簇质心$\mu_j$到其下辖样本集合的中心。下图为$k$-means算法的简单演示：

<img src="./resource/chapter12_image01.png" width="600" alt="" align=center />

图中圆点为训练样本，叉号为簇质心，演示的过程为：(a)原数据集；(b)随机初始化的簇质心（在本例中并不是由某两个随机样本作为簇质心）；(c-f)演示了$k$-means算法的两个迭代，在每次迭代中，我们都将训练样本分配给距离它们最近的簇质心（图中将样本与对应的簇质心标记为同一颜色）；然后将每个簇质心移动到其下辖样本的中心。

那么，$k$-means算法一定收敛吗？在特定的场景下，答案是肯定的。我们先定义**失真函数（distortion function）**：

$$
J(c,\mu)=\sum_{i=1}^m\left\lVert x^{(i)}-\mu_{c^{(i)}}\right\rVert^2
$$

$J$为每个样本$x^{(i)}$到其簇质心$\mu_{c^{(i)}}$的距离的平方之和，可以证明$k$-means正是在$J$上的坐标下降过程（参考[第八讲](chapter08.ipynb)）。尤其是$k$-means算法的内循环实际上是在重复：保持$\mu$不变最小化$J$关于$c$的函数，然后再保持$c$不变最小化$J$关于$\mu$的函数。因此，$J$是单调递减的，它的函数值一定收敛。（通常这意味着$c$和$\mu$也会收敛。但在理论上，$k$-means算法的最终结果可能在少数簇之间振荡，比如少数几组$c$或/且$\mu$具有完全一样的$J$值。不过在实际操作中，这种现象几乎不会发生。）

失真函数$J$是一个非凸函数，所以对于$J$的坐标下降并不能保证其收敛于全局最小值，即$k$-means会被局部最小值影响。尽管如此，$k$-means算法通常都都能很好的完成聚类任务。不过，如果真的怕算法落入“不好的”局部最优中，一个可行的解决方案是为簇质心$\mu_j$赋上不同的初始化值，多运行几次，然后选出失真函数$J(c,\mu)$最小的一组聚类结果即可。

## 混合高斯分布（Mixtures of Gaussians）与最大期望算法（EM algorithm）

在这一节我们将讨论对聚类样本密度估计的最大期望（EM: Expectation-Maximization）。

